Implement svc model

In [1]:
#local file path adjustments
import sys
sys.path.append('C:/Users/Thomas/Desktop/comps/Explainable-Ai-Comps-2024/mooc/src/')

#imports for models
import svc
import mlp
import data_loader

#imports for Anchor
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular

import csv

dl = data_loader.DataLoader()
dl.load(path='C:/Users/Thomas/Desktop/comps/Explainable-Ai-Comps-2024/MOOC/data/')

svc_model = svc.SVC('C:/Users/Thomas/Desktop/comps/Explainable-Ai-Comps-2024/MOOC/src/MITx-SVC.pkl', from_file=True)

mlp_model = mlp.MLPClassifier('C:/Users/Thomas/Desktop/comps/Explainable-Ai-Comps-2024/MOOC/src/MITx-MLP.pkl', from_file=True)

#implementing ANCHOR
MOOC_feature_names = ['viewed', 'gender',   'grade',    'nevents', 'ndays_act', 'nplay_video',  'nchapters',    'age',  'votes',    'num_words'] #thanks Lev

MOOC_categorical_names = {
    0: ['not viewed', 'viewed'],
    1: ['m', 'f']    
}

np.random.seed(1)




c:\Users\Thomas\Desktop\comps\Explainable-Ai-Comps-2024\MOOC\anchor\data_loader.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['gender']] = integer_encoded
C:\Users\Thomas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Thomas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-p

In [4]:

explainer = anchor_tabular.AnchorTabularExplainer(
    ['fail','pass'], #class names
    MOOC_feature_names, #feature names
    dl.X_train, #training data
    MOOC_categorical_names #categorical names)
)


file_path = 'output.csv'
#headers = ['index', 'SVM prediction', 'MLP prediction', 'true_class', 'prediction correct' 'SVM anchor', 'MLP anchor', 'precision', 'coverage'] + MOOC_feature_names
headers = ['index'] + MOOC_feature_names + ['true class', 'SVM prediction', 'SVM anchor', 'SVM precision', 'SVM coverage', 'MLP prediction', 'MLP anchor', 'MLP precision', 'MLP coverage']

with open(file_path, 'w', newline='') as file:
   writer = csv.writer(file)
   writer.writerow(headers)

data = []

indicies = [1,9,26,32,34,22]

for i in indicies:
    pred_svm = explainer.class_names[svc_model.predict(dl.X_test[i].reshape(1, -1))[0]]
    pred_mlp = explainer.class_names[mlp_model.predict(dl.X_test[i].reshape(1, -1))[0]]
        
    
    try:
      exp_svm = explainer.explain_instance(dl.X_test[i], svc_model.predict, threshold=0.99)
      exp_mlp = explainer.explain_instance(dl.X_test[i], mlp_model.predict, threshold=0.99)
    except:
      print ("Error: likley querried beyond list")

    #print('Anchor: %s' % (' AND '.join(exp.names())))
    anchor_svm = ' AND '.join(exp_svm.names())
    anchor_mlp = ' AND '.join(exp_mlp.names())

    precision_svm, coverage_svm = exp_svm.precision(), exp_svm.coverage()
    precision_mlp, coverage_mlp = exp_mlp.precision(), exp_mlp.coverage()

    true_class = '?'

    data.append([i] + list(dl.X_test[i]) + [true_class, pred_svm, anchor_svm, precision_svm, coverage_svm, pred_mlp, anchor_mlp, precision_mlp, coverage_mlp])

    if i % 1 == 0:
        print(i)
        print('\tTrue class: %s' % true_class)
        print('\tSVM prediction: %s' % pred_svm)
        print('\tSVM anchor: %s' % anchor_svm)
        print('\tSVM precision: %.2f' % precision_svm)
        print('\tSVM coverage: %.2f' % coverage_svm)
        print('\tMLP prediction: %s' % pred_mlp)
        print('\tMLP anchor: %s' % anchor_mlp)
        print('\tMLP precision: %.2f' % precision_mlp)
        print('\tMLP coverage: %.2f' % coverage_mlp)
        print('')
        with open(file_path, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(data)
        data = []
        


    
    




# Open the file in write mode ('w') and write the data





PermissionError: [Errno 13] Permission denied: 'output.csv'